# read data 2018-now

In [ ]:
import pandas as pd
import datetime
from datetime import date

data_all=pd.read_csv(r"D:\Masterdata\Clean Data\data_all_20 September With Order Online.csv", sep = ';',converters = {'Phone' : str, 'Order #' : str, 'AWB' : str})

data_all['True datetime']=pd.to_datetime(data_all['True datetime'])
data_all['Real SKU']=data_all['Real SKU'].astype(str)
osf=pd.read_excel(r"D:\Masterdata\order filter.xlsx")
data_all['Phone']=data_all['Phone'].str.extract('(\d+)')
# df['Month_int']=df['True datetime'].dt.month

In [ ]:
df=data_all.copy()

# 2nd tier

In [ ]:
import os
from datetime import date,timedelta

tier2nd=['Tropicana Slim Saus Tiram 200ml','Tropicana Slim Strawberry Jam 375gr','Tropicana Slim Mayonnaise 200 g - Bantu Dukung Hidup Sehat',
         'Tropicana Slim Oat Drink 190 ml (RTD)','Tropicana Slim Shirataki Noodles 71gr','Tropicana Slim Klepon Cookies (5 Sch)',
         'HiLo School RTD Coklat 200ml',"HiLo Teen Taro 500gr",'HiLo Platinum Swiss Chocolate 420gr',"HiLo Gold Sweet Potato 500 gram",
         'HiLo Almond Milk Coconut 200gr','HiLo School Strawberry Cheesecake 500 gram','L-Men Daily Popcorn Caramel 250gr',
         'L-Men Lose Weight Avocado Coffee 300g','L-Men PlantProtein Ogura 216g','L-Men Protein Bar Chocolate 12 sch',
         'L-Men Protein Crunch BBQ Beef (20gr)']

tier2nd=['Tropicana Slim Oat Drink 190 ml (RTD)']
# tier2nd=['Tropicana Slim Saus Tiram 200ml']



os.mkdir(r"\2ndtier_{}".format(date.today()))
df['Month_int']=df['True datetime'].dt.month


for i in tier2nd:
    print(i)
    osf=pd.read_excel(r"order filter.xlsx")
    osf
    ## sales 6 bulan
    init=date.today()
    enddate=init.replace(day=1)
    startdate=init.replace(day=1)-timedelta(1)
    n=0
    while n<5:
        startdate=(startdate.replace(day=1)-timedelta(1)).replace(day=1)
        n=n+1
    sales6m=df[(df['Store Type'].isin(['Marketplace','Organic']))&
               (df['True datetime']>=str(startdate))&(df['True datetime']<str(enddate))&
               (df['Order Status'].isin(osf['order filter'].unique()))&
               (df['Real Nama Produk']==i)].groupby(['Year','Month_int','Month','Store'])[['Total Net Before PPN']].sum().reset_index()
    
    
    # terjual dalam bundle apa / single
    startdate=init.replace(day=1)-timedelta(1)
    dbbndl=df[df['Brand']=='Bundle'][['Store','Product Name','Real SKU']].drop_duplicates()
    bndl=df[(df['Store Type'].isin(['Marketplace','Organic']))&
            (df['Year']==startdate.year)&(df['Month']==startdate.strftime("%B"))&
            (df['Order Status'].isin(osf['order filter'].unique()))&(df['Real Nama Produk']==i)].groupby(['Store','Bundle Name'],dropna=False)[['Qty. Invoiced']].sum().reset_index()

    dbbndl.loc[dbbndl['Real SKU'].str.contains("P"),'Jenis Bundle']='Bundle Sama'
    dbbndl.loc[dbbndl['Real SKU'].str.contains("C"),'Jenis Bundle']='Bundle Sama'
    dbbndl.loc[dbbndl['Real SKU'].str[0]=="P",'Jenis Bundle']='Bundle Beda'
    dbbndl.loc[(dbbndl['Product Name'].str.contains("Buy",case=False))&(dbbndl['Product Name'].str.contains("Get",case=False)),'Jenis Bundle']='Bundle Sama'
    dbbndl.loc[(dbbndl['Product Name'].str.contains("twin|triple|carton",case=False)),'Jenis Bundle']='Bundle Sama'
    
    temp=dbbndl.groupby(['Product Name'])[['Jenis Bundle']].nunique().reset_index()
    temp=temp[temp['Jenis Bundle']>1]['Product Name'].unique()
    dbbndl.loc[(dbbndl['Product Name'].isin(temp)),'Jenis Bundle']='Bundle Sama'
    dbbndl2=dbbndl[['Store','Product Name','Jenis Bundle']].drop_duplicates()
    
    # display(bndl)
    # print(bndl['Bundle Name'].nunique())
    
    if bndl['Bundle Name'].nunique() <1 :
        bndl['Jenis Bundle']='Single'
        bndl2=bndl.copy()
    else:
        #ada merge
        bndl2=bndl.merge(dbbndl2[['Store','Product Name','Jenis Bundle']].rename(columns={'Product Name':'Bundle Name'}),how='left')
    
    # display(bndl2)
    
    bndl2.loc[bndl2['Jenis Bundle'].isnull(),'Jenis Bundle']='Single'
    bndl2=bndl2.groupby(['Store','Jenis Bundle'])[['Qty. Invoiced']].sum().reset_index().pivot(index='Store',columns='Jenis Bundle')['Qty. Invoiced'].reset_index()

    init=date.today()
    enddate=init.replace(day=1)-timedelta(1)
    startdate=(init.replace(day=1)-timedelta(1)).replace(day=1)-timedelta(1)

    startdate.year
    startdate.strftime("%B")

    a=df[(df['Store Type'].isin(['Marketplace','Organic']))&
               (df['Year']==startdate.year)&(df['Month']==startdate.strftime("%B"))&
               (df['Order Status'].isin(osf['order filter'].unique()))&
               (df['Real Nama Produk']==i)].groupby(['Store'])[['Selling Price']].median().reset_index().rename(columns={'Selling Price':'SP {}'.format(startdate.strftime("%B"))})
    b=df[(df['Store Type'].isin(['Marketplace','Organic']))&
               (df['Year']==startdate.year)&(df['Month']==startdate.strftime("%B"))&
               (df['Order Status'].isin(osf['order filter'].unique()))&
               (df['Real Nama Produk']==i)].groupby(['Store'])[['Qty. Invoiced']].sum().reset_index().rename(columns={'Qty. Invoiced':'Sales {}'.format(startdate.strftime("%B"))})
    c=df[(df['Store Type'].isin(['Marketplace','Organic']))&
               (df['Year']==enddate.year)&(df['Month']==enddate.strftime("%B"))&
               (df['Order Status'].isin(osf['order filter'].unique()))&
               (df['Real Nama Produk']==i)].groupby(['Store'])[['Selling Price']].median().reset_index().rename(columns={'Selling Price':'SP {}'.format(enddate.strftime("%B"))})
    d=df[(df['Store Type'].isin(['Marketplace','Organic']))&
               (df['Year']==enddate.year)&(df['Month']==enddate.strftime("%B"))&
               (df['Order Status'].isin(osf['order filter'].unique()))&
               (df['Real Nama Produk']==i)].groupby(['Store'])[['Qty. Invoiced']].sum().reset_index().rename(columns={'Qty. Invoiced':'Sales {}'.format(enddate.strftime("%B"))})
    dfsp=a.merge(b).merge(c).merge(d)
    PL=df[(df['Store Type'].isin(['Marketplace','Organic']))&
       (df['Year']==enddate.year)&(df['Month']==enddate.strftime("%B"))&
       (df['Order Status'].isin(osf['order filter'].unique()))&
       (df['Real Nama Produk']==i)]['Price List NFI'].median()
    dfsp['BTL']=(dfsp['SP {}'.format(enddate.strftime("%B"))]/PL)-1
    dfsp['Growth']=dfsp['Sales {}'.format(enddate.strftime("%B"))]/dfsp['Sales {}'.format(startdate.strftime("%B"))]-1

    temp=df[(df['Year']==enddate.year)&(df['Month']==enddate.strftime("%B"))&(df['Real Nama Produk']==i)&(df['Store Type'].isin(['Marketplace', 'Organic']))].groupby(['Store'])[['Qty. Invoiced']].mean().reset_index()
    # print(df[(df['Year']==enddate.year)&(df['Month']==enddate.strftime("%B"))&(df['Real Nama Produk']==i)&(df['Store Type'].isin(['Marketplace', 'Organic']))&(df['Qty. Invoiced']==1)][['Total Net']].describe())
    ord=df[(df['Year']==enddate.year)&(df['Month']==enddate.strftime("%B"))&(df['Real Nama Produk']==i)&(df['Store Type'].isin(['Marketplace', 'Organic']))]['Order #'].unique()
    temp2=df[(df['Order #'].isin(ord))&(df['Brand'].isin(['L-Men','TS','HiLo', 'NS','WDANK']))].groupby(['Store','Order #'])[['Total Net']].sum().reset_index().groupby(['Store'])[['Total Net']].mean().reset_index()
    temp=temp.merge(temp2,how='left',on='Store')
    temp2=df[(df['Order #'].isin(ord))&(df['Brand'].isin(['L-Men','TS','HiLo', 'NS','WDANK']))&(df['Store Type'].isin(['Marketplace', 'Organic']))].groupby(['Real Nama Produk'])[['Order #']].nunique().reset_index().sort_values('Order #',ascending=False).head(11)


    with pd.ExcelWriter(r"2ndtier_{}\{}.xlsx".format(date.today(),i)) as writer:  
        sales6m.to_excel(writer, sheet_name="tbl1",index=False)
        bndl2.to_excel(writer, sheet_name="tbl2",index=False)
        dfsp.to_excel(writer, sheet_name="tbl3",index=False)
        temp.to_excel(writer, sheet_name="tbl4",index=False)
        temp2.to_excel(writer, sheet_name="tbl5",index=False)


    print('done')
print('done')

## pantau balapan

In [ ]:
from datetime import date,timedelta

init=date.today()
enddate=init.replace(day=1)
startdate=date.today().replace(day=1)-timedelta(1)
n=0
while n<5:
    startdate=(startdate.replace(day=1)-timedelta(1)).replace(day=1)
    n=n+1
sl=['Nutrimart', 'Blibli', 'Lazada', 'Tokopedia', 'Bukalapak','JD Indonesia', 'Shopee','TikTok']
oil=['Tropicana Slim Minyak Kanola 946ml','Tropicana Slim Minyak Jagung 946ml','Tropicana Slim Sunflower Oil 946 ml','Tropicana Slim Extra Virgin Olive Oil 500 ml']
cookies=['Tropicana Slim Klepon Cookies (5 Sch)',
         'Tropicana Slim Hokkaido Cheese Cookies',
         'Tropicana Slim Korean Goguma Cookies (5 Sch)',
         'Tropicana Slim Nutty Chocolate Cookies 200gr',
         'Tropicana Slim Korean Garlic Butter Cookies 5 Sch']
kopi=['Tropicana Slim Cafe Latte (10 sch)',
      'Tropicana Slim White Coffee',
      'Tropicana Slim Avocado Coffee 4 Sch']
teen500=['HiLo Teen Taro 500gr','HiLo Teen Vanilla Caramel 500gr','Ex Shopee - HiLo Teen Chocolate 500gr','HiLo Teen Chocolate 500gr','HiLo Teen Popcorn Caramel 500 gram - Susu Tinggi Kalsium Lebih Rendah Lemak',"HiLo Teen Strawberry Milkshake 500g",'HiLo Teen Biscuit Caramel 500gr']
# active=['2101452180P2',"2101452195"]
school=['HiLo School Strawberry Cheesecake 500 gram','HiLo School Chocolate 500gr', 'HiLo School Honey 500gr','HiLo School Cotton Candy 500g','HiLo School Bubble Gum 500g','HiLo School Vanilla Vegiberi 500gr']
plt=['L-Men Platinum Bubble Gum 800 gram','L-Men Platinum Choco Latte 800gr (25gr protein)','L-Men Platinum Kacang Hijau 800g','L-Men Platinum Ketan Hitam 800g (25g protein / serving) - Suplemen Tinggi Whey Protein Rendah Lemak','L-Men Platinum Bubble Gum 800 gram']
dayli=['L-Men Whey Daily Dark Chocolate 250gr',
       'L-Men Daily Popcorn Caramel 250gr']
bumbu=['Tropicana Slim Kecap Manis 200ml',
       'Tropicana Slim Kecap Asin 200 ml',
       'Tropicana Slim Saus Tiram 200ml',
       'Tropicana Slim Mayonnaise 200 g - Bantu Dukung Hidup Sehat']
lose=['L-Men Lose Weight Chocolate Cereal 300gr',
       'L-Men Lose Weight Avocado Coffee 300g',
       'L-Men Lose Weight Mango Sticky Rice 300gr']

for i in [oil,cookies,kopi,teen500,school,plt,dayli,bumbu,lose]:
# for i in [teen500,school]:
    
# for i in [plt,lose]:
    print(i)
    res1=df[(df['Real Nama Produk'].isin(i))&(df['True datetime']>=str(startdate))&(df['True datetime']<str(enddate))&
                 (df['Store'].isin(sl))&(df['Order Status'].isin(osf['order filter'].unique()))].groupby(['Year','Month','Month_int','Parent Item'])[['Total Net Before PPN']].sum().reset_index()
    col=res1.pivot(columns='Parent Item',index=['Year','Month_int','Month'])['Total Net Before PPN'].reset_index().columns.to_list()
    col.remove('Month_int')
    res1=res1.pivot(columns='Parent Item',index=['Year','Month_int','Month'])['Total Net Before PPN'].reset_index()[col]

    lastmonth=date.today().replace(day=1)-timedelta(1)
    res2=df[(df['Real Nama Produk'].isin(i))&(df['Year']==lastmonth.year)&(df['Month']==lastmonth.strftime("%B"))&
            (df['Store'].isin(sl))&(df['Order Status'].isin(osf['order filter'].unique()))].groupby(['Store','Parent Item'])[['Total Net Before PPN']].sum().reset_index().pivot(columns='Store',index='Parent Item')['Total Net Before PPN'].reset_index()
    with pd.ExcelWriter(r"{}{}.xlsx".format(date.today(),i[0])) as writer:  
        res1.to_excel(writer, sheet_name="tbl1_1",index=False)
        res2.to_excel(writer, sheet_name="tbl1_2",index=False)
print("done")